# Phase 3: QA Model Development

In [39]:
import openai
import pinecone
import sys

print("OpenAI version:", openai.__version__)
print("Pinecone version:", pinecone.__version__)
print("Python version:", sys.version)

OpenAI version: 2.8.1
Pinecone version: 8.0.0
Python version: 3.13.7 (main, Aug 14 2025, 11:12:11) [Clang 17.0.0 (clang-1700.0.13.3)]


In [40]:
!pip install langchain

/Users/sofiazogkza/repos/Ironhack/ChainMind/venvSofia/bin/pip: line 2: /Users/sofiazogkza/repos/Ironhack/Ironhack Final Project/venvSofia/bin/python3.13: No such file or directory
/Users/sofiazogkza/repos/Ironhack/ChainMind/venvSofia/bin/pip: line 2: exec: /Users/sofiazogkza/repos/Ironhack/Ironhack Final Project/venvSofia/bin/python3.13: cannot execute: No such file or directory


In [23]:
from openai import OpenAI
from pinecone import Pinecone
import os

# ------------------------------
# Environment variables
# ------------------------------
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV", "us-east-1")  # not used by new SDK but kept
index_name = "youtube-chunks"

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ------------------------------
# Pinecone initialization
# ------------------------------
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(index_name)

# ------------------------------
# Conversation memory
# ------------------------------
conversation_history = []  # stores last turns


# ------------------------------
# 1) Retrieve top chunks from pinecone
# ------------------------------
def retrieve_from_pinecone(query, k=3):
    query_embedding = client.embeddings.create(
        model="text-embedding-3-small",
        input=query
    ).data[0].embedding

    results = index.query(
        vector=query_embedding,
        top_k=k,
        include_metadata=True
    )

    contexts = [m.metadata["text_chunk"] for m in results.matches]
    return "\n\n".join(contexts)


# ------------------------------
# 2) LLM QA with memory
# ------------------------------
def answer_question(query):
    # Get context from Pinecone
    context = retrieve_from_pinecone(query)

    # Build conversation memory (last 5 turns)
    memory_text = ""
    for turn in conversation_history[-5:]:
        memory_text += f"USER: {turn['question']}\nASSISTANT: {turn['answer']}\n\n"

    prompt = f"""
You are a helpful assistant answering questions about YouTube videos.

You have TWO sources:

1) Conversation Memory (for conversational continuity ONLY)
{memory_text}

2) Video Context (for factual information — ALWAYS use this for answering questions)
{context}

QUESTION: {query}

If the answer is not in the context but is in the conversation memory, use the memory.
If it is in neither, say: "The video does not explain this clearly."
"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
    )

    answer = response.choices[0].message.content

    # Save new turn to memory
    conversation_history.append({
        "question": query,
        "answer": answer
    })

    return answer


# 🧪 TESTING

In [45]:
question = "How do I create a smart contract for a blockchain game?"
answer = answer_question(question)
print("Question:", question)
print("Answer:", answer)

Question: How do I create a smart contract for a blockchain game?
Answer: To create a smart contract for a blockchain game, you would need to follow these steps: 
1. Learn the basics of smart contracts.
2. Set up a development environment.
3. Create an NFT smart contract.
4. Create a wagering smart contract.
5. Deploy the smart contracts.
6. Compile and display the smart contracts.


In [25]:
question = "Why should I create a game based on blockchain?"
answer = answer_question(question)
print("Question:", question)
print("Answer:", answer)

Question: Why should I create a game based on blockchain?
Answer: The video does not explain this clearly.


In [12]:
question = "Give me an example!"
answer = answer_question(question)
print("Question:", question)
print("Answer:", answer)

Question: Give me an example!
Answer: The video does not explain this clearly.


In [13]:
question = "what was the previous example?"
answer = answer_question(question)
print("Question:", question)
print("Answer:", answer)

Question: what was the previous example?
Answer: The video does not explain this clearly.


Result: 

- ❌ Hallucinations: Or GPT-3.5-turbo is too weak and ignores context when it's vague
- We can observe that there is not memory to remember previous questions.

Debug:
- Added # DEBUG code in the previous codeblock & re-run the QAs.

Solutions:
- top_k from 3 to 34 (which are all the chunks).
- added `summarize_chunk: Each chunk is summarized into 2-3 sentences.
- summarizing each chunk on the fly -> But it made it veeeery SLOW. -> SOLUTION: Summarize chunks once | and store as `summar`y` in Pinecone metadata

# 🔗 Setup LangChain + Pinecone retriever

In [35]:
!pip install -qU langchain-pinecone langchain-openai



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [55]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone

# Use your OpenAI API key
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Wrap Pinecone index as LangChain vectorstore
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embeddings,
    text_key="text_chunk"
)


### Create a **RetrievalQA** chain

In [62]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.2,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# Create retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Define prompt template
template = """Answer the question based on the following context from video transcripts:

Context: {context}

Question: {question}

Answer:"""

prompt = ChatPromptTemplate.from_template(template)

# Build the RAG chain - format docs from retriever properly
rag_chain = (
    {
        "context": retriever | RunnableLambda(lambda docs: "\n\n".join([doc.page_content for doc in docs])),
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Use it to answer questions
response = rag_chain.invoke("How to build a blockchain game?")
print(response)

To build a blockchain game, you can follow these steps:

1. Start by using the third web CLI to create a smart contract for your game.
2. Choose a smart contract framework like Hardhat or Forge.
3. Develop the smart contract for your game, including functions for authentication, querying NFTs, depositing NFTs into a wagering smart contract, and withdrawing winnings.
4. Implement the front end code to interact with the Ethereum blockchain.
5. Create a user interface for wallet interaction.
6. Test the game to ensure it functions properly.
7. Promote the game to attract players.
8. Consider giving away NFTs for free to promote real ownership in blockchain gaming.
9. Make sure to focus on creating high-quality projects and changing the perception of blockchain gaming.


Observations:
- `RetrievalQA` was deprecated in LangChain 1.0+ because it's less flexible than building chains manually with runnables.

In [53]:
question = "How to build a blockchain game?"
print("Question:", question)
print("Answer:", end=" ", flush=True)

for chunk in rag_chain.stream(question):
    print(chunk, end="", flush=True)
print()

Question: How to build a blockchain game?
Answer: To build a blockchain game, you can follow these steps:

1. Learn the basics of blockchain technology and smart contracts.
2. Set up a development environment for blockchain development.
3. Create an NFT smart contract for your game.
4. Create a wagering smart contract for players to enter the game.
5. Deploy the smart contracts to a local test blockchain.
6. Implement the front end code to interact with the Ethereum blockchain.
7. Create functions to authenticate users, query NFTs, deposit NFTs into the wagering smart contract, and withdraw winnings.
8. Create a user interface for wallet interaction and game play.
9. Test the game and promote it to users.

By following these steps, you can create a blockchain game with real ownership and unique gameplay mechanics.


In [54]:
question = "How to reate an NFT smart contract for my game?"
print("Question:", question)
print("Answer:", end=" ", flush=True)

for chunk in rag_chain.stream(question):
    print(chunk, end="", flush=True)
print()

Question: How to reate an NFT smart contract for my game?
Answer: To create an NFT smart contract for your game, you can follow these steps:

1. Learn the basics of smart contract development.
2. Set up a development environment.
3. Create the NFT smart contract using a tool like gbt4.
4. Deploy the smart contract to a local test blockchain.
5. Connect the NFT smart contract with your game to authenticate users, query NFTs, deposit NFTs into the wagering smart contract, and withdraw winnings.
6. Implement the front end code to interact with the Ethereum blockchain.
7. Test the game to ensure it functions properly.
8. Create a user interface for wallet interaction.
9. Promote the game to attract players.

By following these steps, you can successfully create an NFT smart contract for your game.


# MEMORY + INTERNET SEARCH AGENT

In [82]:
# ======================================================
# INTERNET SEARCH AGENT WITH MEMORY + FOLLOW-UP HANDLING
# ======================================================

from openai import OpenAI
from serpapi import GoogleSearch
import json
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ----------------------------
# GLOBAL MEMORY
# ----------------------------
conversation_history = []


# ======================================================
# INTERNET SEARCH TOOL
# ======================================================
def internet_search(query):
    params = {
        "q": query,
        "api_key": os.getenv("SERPAPI_KEY")
    }
    search = GoogleSearch(params)
    result = search.get_dict()

    if "organic_results" in result and len(result["organic_results"]) > 0:
        return "\n".join([item.get("snippet", "") for item in result["organic_results"][:3]])

    return "No results found."


# ======================================================
# MAIN AGENT FUNCTION
# ======================================================
def agent_with_search(query, rag_answer):
    global conversation_history

    # ----------------------------
    # Build memory context text
    # ----------------------------
    memory_text = ""
    for turn in conversation_history[-5:]:
        memory_text += f"USER: {turn['question']}\nASSISTANT: {turn['answer']}\n\n"

    # ======================================================
    # 0) FOLLOW-UP QUESTION DETECTION
    # ======================================================
    follow_up_phrases = [
        "give me an example",
        "another example",
        "what was the previous",
        "explain again",
        "repeat that",
        "continue",
        "elaborate",
        "what did you say before",
        "remind me",
        "again please"
    ]

    lower_q = query.lower()

    # If this is a follow-up question AND we have memory → return last answer
    if any(phrase in lower_q for phrase in follow_up_phrases):
        if len(conversation_history) > 0:
            final_answer = conversation_history[-1]["answer"]

            # Save follow-up question to memory too
            conversation_history.append({
                "question": query,
                "answer": final_answer
            })
            return final_answer


    # ======================================================
    # 1) RAG KNOWS THE ANSWER → ACCEPT IT
    # ======================================================
    if rag_answer.strip() != "The video does not explain this clearly.":
        conversation_history.append({
            "question": query,
            "answer": rag_answer
        })
        return rag_answer


    # ======================================================
    # 2) RAG FAILED → TRY INTERNET SEARCH
    # ======================================================
    tools = [
        {
            "type": "function",
            "function": {
                "name": "internet_search",
                "description": "Search the web for real-time information.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {"type": "string"}
                    },
                    "required": ["query"]
                }
            }
        }
    ]

    # Ask the model whether to use the tool
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system",
             "content": (
                 "You are an intelligent agent.\n"
                 "Use MEMORY for context, but if the video lacks the answer, "
                 "call the internet_search tool.\n"
             )},
            {"role": "assistant", "content": f"MEMORY:\n{memory_text}"},
            {"role": "user", "content": query}
        ],
        tools=tools,
        tool_choice="auto"
    )

    message = response.choices[0].message

    # ======================================================
    # 3) TOOL CALL → INTERNET SEARCH
    # ======================================================
    if message.tool_calls:
        tool_call = message.tool_calls[0]
        args = json.loads(tool_call.function.arguments)
        search_query = args.get("query", query)

        search_output = internet_search(search_query)

        # Now rewrite search results into a helpful answer
        refined = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Rewrite the search results into a clean, helpful answer."},
                {"role": "assistant", "content": f"Search results:\n{search_output}"},
                {"role": "user", "content": query}
            ]
        )

        final_answer = refined.choices[0].message.content

        # Save memory
        conversation_history.append({
            "question": query,
            "answer": final_answer
        })

        return final_answer

    # ======================================================
    # 4) NO TOOL CALL → fallback to RAG answer
    # ======================================================
    conversation_history.append({
        "question": query,
        "answer": rag_answer
    })
    return rag_answer


In [110]:
query = "How do I create a smart contract for a blockchain game?"
print("Question:", query)
# 1. Get the RAG answer first
rag_answer = answer_question(query)
print("RAG Answer:", rag_answer)

# 2. Use the agent to decide whether to search the internet
final_answer = agent_with_search(query, rag_answer)
print("\nAgent Answer:", final_answer)


Question: How do I create a smart contract for a blockchain game?
RAG Answer: MEMORY: To create a smart contract for a blockchain game, you can use the third web CLI to create a smart contract as the backend engine for your game. Choose a smart contract framework like Hardhat or Forge, and follow step-by-step instructions to create the necessary components such as NFT smart contracts and wagering smart contracts. Additionally, deploy the smart contracts to a local test blockchain and implement front-end code to interact with the Ethereum blockchain. This process will help you get started on creating a smart contract for a blockchain game.

Agent Answer: MEMORY: To create a smart contract for a blockchain game, you can use the third web CLI to create a smart contract as the backend engine for your game. Choose a smart contract framework like Hardhat or Forge, and follow step-by-step instructions to create the necessary components such as NFT smart contracts and wagering smart contracts.

In [111]:
query = "Why should I create a game based on blockchain?"
print("Question:", query)

rag_answer = answer_question(query)
print("\nRAG Answer:", rag_answer)
final_answer = agent_with_search(query, rag_answer)
print("\nAgent Answer:", final_answer)


Question: Why should I create a game based on blockchain?

RAG Answer: MEMORY: Creating a game based on blockchain allows for real ownership of in-game assets, ensuring that players truly own the items they acquire in the game. This can help combat scams and provide a new level of transparency and security in gaming.

Agent Answer: MEMORY: Creating a game based on blockchain allows for real ownership of in-game assets, ensuring that players truly own the items they acquire in the game. This can help combat scams and provide a new level of transparency and security in gaming.


In [112]:
query = "Give me an example!"
print("Question:", query)
rag_answer = answer_question(question)
print("\nRAG Answer:", rag_answer)
final_answer = agent_with_search(question, rag_answer)
print("\nAgent Answer:", final_answer)

Question: Give me an example!

RAG Answer: MEMORY: The previous example was about customizing a smart contract for a blockchain game to change the logic of burning NFTs and minting new tokens when a specific action is performed.

Agent Answer: MEMORY: The previous example was about customizing a smart contract for a blockchain game to change the logic of burning NFTs and minting new tokens when a specific action is performed.


In [113]:
query = "What was the previous example?"
print("Question:", query)
rag_answer = answer_question(question)
print("\nRAG Answer:", rag_answer)
final_answer = agent_with_search(question, rag_answer)
print("\nAgent Answer:", final_answer)

Question: What was the previous example?

RAG Answer: MEMORY: The previous example was about customizing a smart contract for a blockchain game to change the logic of burning NFTs and minting new tokens when a specific action is performed.

Agent Answer: MEMORY: The previous example was about customizing a smart contract for a blockchain game to change the logic of burning NFTs and minting new tokens when a specific action is performed.


In [108]:
query = "What is wagering smart contract?"
print("Question:", query)

rag_answer = answer_question(query)
print("\nRAG Answer:", rag_answer)
final_answer = agent_with_search(query, rag_answer)
print("\nAgent Answer:", final_answer)


Question: What is wagering smart contract?

RAG Answer: MEMORY: The wagering smart contract for a game interacts with the previously created NFT contract and allows players to deposit their tokens, create matches, determine winners and losers, and transfer tokens to the winner.

Agent Answer: MEMORY: The wagering smart contract for a game interacts with the previously created NFT contract and allows players to deposit their tokens, create matches, determine winners and losers, and transfer tokens to the winner.
